In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [25]:
import numpy as np
import pandas as pd

data = pd.read_csv("customers.csv")

data.head()

,Gender_x,Age,Profession,Work_Experience,Family_Size,Spending_Score,Cost_of_the_Product,Prior_purchases,Discount_offered
0,Male,22,Healthcare,1.0,4.0,Low,177,3,44
1,Female,38,Engineer,0.0,3.0,Average,216,2,59
2,Female,67,Engineer,1.0,1.0,Low,183,4,48
3,Male,67,Lawyer,0.0,2.0,High,176,4,10
4,Female,40,Entertainment,0.0,6.0,High,184,3,46


In [26]:
# string to int classification
from sklearn.preprocessing import LabelEncoder

mapping_string_to_int = {}

label_encoder = LabelEncoder()
string_fields = ['Gender_x', 'Profession', 'Spending_Score']
for string_field in string_fields:
    old = data[string_field].copy()
    data[string_field] = label_encoder.fit_transform(data[string_field])
    new = data[string_field]
    pairs = zip(old, new)


    for key, val in pairs:
        mapping_string_to_int[key] = val



data.head()

,Gender_x,Age,Profession,Work_Experience,Family_Size,Spending_Score,Cost_of_the_Product,Prior_purchases,Discount_offered
0,1,22,5,1.0,4.0,2,177,3,44
1,0,38,2,0.0,3.0,0,216,2,59
2,0,67,2,1.0,1.0,2,183,4,48
3,1,67,7,0.0,2.0,1,176,4,10
4,0,40,3,0.0,6.0,1,184,3,46


In [27]:
# data normalization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
mins_and_ranges = []

for field_name in data.columns:
    data[field_name] = scaler.fit_transform(data[[field_name]])
    mins_and_ranges.append((scaler.data_min_, scaler.data_range_))

print(data)
print(mins_and_ranges)



      Gender_x       Age  Profession  Work_Experience  Family_Size  \
0          1.0  0.056338       0.625         0.071429     0.444444   
1          0.0  0.281690       0.250         0.000000     0.333333   
2          0.0  0.690141       0.250         0.071429     0.111111   
3          1.0  0.690141       0.875         0.000000     0.222222   
4          0.0  0.309859       0.375         0.000000     0.666667   
...        ...       ...         ...              ...          ...   
7939       1.0  0.323944       0.000         0.000000     0.555556   
7940       1.0  0.239437       0.500         0.214286     0.444444   
7941       0.0  0.211268       0.625         0.071429     0.111111   
7942       0.0  0.126761       0.625         0.071429     0.444444   
7943       1.0  0.267606       0.500         0.000000     0.333333   

      Spending_Score  Cost_of_the_Product  Prior_purchases  Discount_offered  
0                1.0             0.378505            0.125          0.671875  
1

In [40]:
from typing import List

def map_customer(customer: List):
    strings_fields_idx = [0, 2]
    l = len(customer)
    for i in strings_fields_idx:
        customer[i] = mapping_string_to_int[customer[i]]

    for i in range(len(customer)):
        customer[i] -= mins_and_ranges[i][0]
        customer[i] /= mins_and_ranges[i][1]

    customer = torch.tensor(customer).to(torch.float32)
    return customer.reshape((1, l))

In [29]:
# test training splitting
from sklearn.model_selection import train_test_split

y_cols = ['Cost_of_the_Product', "Spending_Score", "Discount_offered"]

X = data.drop(columns=y_cols)
y = data[y_cols]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = torch.tensor(X_train.values)
X_test = torch.tensor(X_test.values)
y_train = torch.tensor(y_train.values)
y_test = torch.tensor(y_test.values)

X_train = X_train.to(torch.float32)
X_test = X_test.to(torch.float32)
y_train = y_train.to(torch.float32)
y_test = y_test.to(torch.float32)

print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)
print(X_train.dtype)
print(X_test.dtype)
print(y_train.dtype)
print(y_test.dtype)

Training set shape: torch.Size([6355, 6]) torch.Size([6355, 3])
Testing set shape: torch.Size([1589, 6]) torch.Size([1589, 3])
torch.float32
torch.float32
torch.float32
torch.float32


In [30]:
# NN 1
from torch import sigmoid
class SpendingsPredictor(nn.Module):
    def __init__(self, input_size: int, dropout_p: float = 0.5):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout_p),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        return self.mlp(x)

    def predict_proba(self, x):
        return sigmoid(self(x))

    def predict(self, x):
        y_pred_score = self.predict_proba(x)
        return torch.argmax(y_pred_score, dim=1)

In [31]:
# NN
from torch import sigmoid
class SpendingsPredictor2(nn.Module):
    def __init__(self, input_size: int):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        return self.mlp(x)

    def predict_proba(self, x):
        return sigmoid(self(x))

    def predict(self, x):
        y_pred_score = self.predict_proba(x)
        return torch.argmax(y_pred_score, dim=1)

In [32]:
# training configs

from copy import deepcopy

from torch.utils.data import DataLoader


learning_rate = 1e-3
dropout_p = 0.2
l2_reg = 1e-8
max_epochs = 1000


early_stopping_patience = 20

In [33]:
model = SpendingsPredictor(
    input_size=X_train.shape[1],
    dropout_p=dropout_p
)
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=learning_rate,
    weight_decay=l2_reg
)

# loss_fn = nn.L1Loss()
loss_fn = nn.MSELoss()


In [34]:
def train_model(model):
    steps_without_improvement = 0

    best_val_loss = np.inf
    best_model = None

    for epoch_num in range(max_epochs):
        model.train()

        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        model.eval()

        if loss < best_val_loss:
            best_val_loss = loss
            best_model = deepcopy(model)
            steps_without_improvement = 0
        else:
            steps_without_improvement += 1
        if epoch_num % 50 == 0:
            print(f"Epoch {epoch_num} train loss: {loss.item():.4f}")

        if steps_without_improvement >= early_stopping_patience:
            print(f"Early stopping after {epoch_num} epochs.")
            break

    return best_model

In [35]:

best = train_model(model)

Epoch 0 train loss: 0.3712
Epoch 50 train loss: 0.2991
Epoch 100 train loss: 0.2463
Epoch 150 train loss: 0.2079
Epoch 200 train loss: 0.1777
Epoch 250 train loss: 0.1573
Epoch 300 train loss: 0.1420
Epoch 350 train loss: 0.1319
Epoch 400 train loss: 0.1242
Epoch 450 train loss: 0.1194
Epoch 500 train loss: 0.1160
Epoch 550 train loss: 0.1141
Epoch 600 train loss: 0.1123
Epoch 650 train loss: 0.1125
Early stopping after 692 epochs.


In [36]:
model2 = SpendingsPredictor2(
    input_size=X_train.shape[1],
)
optimizer = torch.optim.SGD(
    model2.parameters(),
    lr=learning_rate,
    weight_decay=l2_reg
)

In [37]:
# best = train_model(model2)

In [41]:
customer = ["Male", 22, "Healthcare", 1.0, 4.0, 3]
customer = map_customer(customer)
best(customer)


tensor([[0.6242, 0.8525, 0.1998]], grad_fn=<AddmmBackward0>)

In [4]:
SAVE_PATH = "spendings_predictor.pt"
torch.save(best, SAVE_PATH)

NameError: name 'best' is not defined

In [5]:
best2 = torch.load(SAVE_PATH)
best2.eval()
customer = ["Male", 22, "Healthcare", 1.0, 4.0, 3]
customer = map_customer(customer)
best2(customer)

AttributeError: Can't get attribute 'SpendingsPredictor' on <module '__main__'>